## 1. 모든 단어 사용

### 데이터 로드

In [50]:
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score #정확도 계산

In [51]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [52]:
print('훈련 데이터 개수:', len(x_train))
print('데스트 데이터 개수:', len(x_test))

훈련 데이터 개수: 8982
데스트 데이터 개수: 2246


In [53]:
# 데이터 확인
print(x_train[0])
print(x_test[0])

[1, 27595, 28842, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
[1, 4, 1378, 2025, 9, 697, 4622, 111, 8, 25, 109, 29, 3650, 11, 150, 244, 364, 33, 30, 30, 1398, 333, 6, 18292, 159, 9, 1084, 363, 13, 19231, 71, 9, 16273, 71, 117, 4, 225, 78, 206, 10, 9, 1214, 8, 4, 270, 5, 16273, 7, 748, 48, 9, 19231, 7, 207, 1451, 966, 1864, 793, 97, 133, 336, 7, 4, 493, 98, 273, 104, 284, 25, 39, 338, 22, 905, 220, 3465, 644, 59, 20, 6, 119, 61, 11, 15, 58, 579, 26, 10, 67, 7, 4, 738, 98, 43, 88, 333, 722, 12, 20, 6, 19, 746, 35, 15, 10, 9, 1214, 855, 129, 783, 21, 4, 2280, 244, 364, 51, 16, 299, 452, 16, 515, 4, 99, 29, 5, 4, 364, 281, 48, 10, 9, 1214, 23, 644, 47, 20, 324, 27, 56, 23406, 28185, 5, 192,

### 데이터 복원하기

In [54]:
word_index = reuters.get_word_index(path="reuters_word_index.json")
word_index['the']

1

reuters.get_word_index에 +3을 해주어야 학습이 올바르게 이루어질 수 있습니다. 또한 필요한 것은 index:word와 같은 형식이므로 변환해 주겠습니다.

In [55]:
index_to_word = {index+3:word for word, index in word_index.items()}
index_to_word[4]

'the'

index_to_word에서 1,2,3 index는 비어 있으므로 적합한 토큰을 추가해줍니다.

In [56]:
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
    index_to_word[index]=token
print(index_to_word[2])

<unk>


훈련, 테스트 데이터를 텍스트 데이터로 변환하겠습니다.

In [57]:
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)
    
x_train = decoded
print(len(x_train))

8982


In [58]:
x_train[0]

'<sos> mcgrath rentcorp said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3'

In [59]:
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)
        
x_test = decoded
print(len(x_test))        

2246


In [60]:
x_test[0]

'<sos> the great atlantic and pacific tea co said its three year 345 mln dlr capital program will be be substantially increased to accommodate growth and expansion plans for waldbaum inc and shopwell inc over the next two years a and p said the acquisition of shopwell in august 1986 and waldbaum in december helped us achieve better than expected results in the fourth quarter ended february 28 its net income from continuing operations jumped 52 6 pct to 20 7 mln dlrs or 55 cts a share in the latest quarter as sales increased 48 3 pct to 1 58 billion dlrs a and p gave no details on the expanded capital program but it did say it completed the first year of the program during 1986 a and p is 52 4 pct owned by lt tengelmann warenhandelsgesellschaft of west germany reuter 3'

### 복원 데이터를 다시 벡터화 하기

In [61]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [62]:
dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

(8982, 26506)


In [63]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

(8982, 26506)


In [65]:
x_test_dtm = dtmvector.transform(x_test)
tfidfv_test = tfidf_transformer.transform(x_test_dtm)
print(tfidfv.shape)

(8982, 26506)


### 나이브 베이즈 분류기(Multinomial Naive Bayes Classifier)

In [66]:
model = MultinomialNB()
model.fit(tfidfv, y_train)

MultinomialNB()

In [67]:
x_test_dtm = dtmvector.transform(x_test) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환

predicted = model.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.5997328584149599


### Complement Naive Bayes Classifier(CNB)

In [68]:
cb = ComplementNB()
cb.fit(tfidfv, y_train)

ComplementNB()

In [69]:
predicted = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7649154051647373


### 로지스틱 회귀(Logistic Regression)

In [70]:
lr = LogisticRegression(C=10000, penalty='l2')
lr.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000)

In [71]:
predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.813446126447017


### 선형 서포트 벡터 머신(Linear Support Vector Machine)

In [72]:
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False)
lsvc.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1000, dual=False, max_iter=500, penalty='l1')

In [73]:
predicted = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7796081923419412


### 결정 트리(Decision Tree)

In [74]:
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [75]:
predicted = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6211041852181657


### 랜덤 포레스트(Random Forest)

In [76]:
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [77]:
predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6544968833481746


### 그래디언트 부스팅 트리(GradientBoostingClassifier)

In [78]:
grbt = GradientBoostingClassifier(random_state=0) # verbose=3
grbt.fit(tfidfv, y_train)

GradientBoostingClassifier(random_state=0)

In [79]:
predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7702582368655387


### 보팅(Voting)

In [80]:
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=10000)),
                             ('cb', ComplementNB()),
                             ('grbt',
                              GradientBoostingClassifier(random_state=0))],
                 n_jobs=-1, voting='soft')

In [81]:
predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8187889581478184


### RNN(LSTM)

In [239]:
# 모듈 import

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [218]:
(x_train_rnn, y_train_rnn), (x_test_rnn, y_test_rnn) = reuters.load_data(num_words=None, test_split=0.2)

In [219]:
num_classes = max(y_train_rnn)+1
num_classes

46

In [220]:
len(index_to_word)

30982

In [221]:
vocab_size = len(index_to_word)
tag_size = num_classes

In [222]:
print('단어 사전의 개수:', vocab_size)
print('레이블의 개수:', tag_size)

단어 사전의 개수: 30982
레이블의 개수: 46


In [223]:
total_data = len(x_train_rnn) + len(x_test_rnn)
total_data

11228

In [224]:
total_data = len(x_train_rnn) + len(x_test_rnn)
print('훈련 문장의 최대 길이:', max(len(i) for i in x_train_rnn))
print('테스트 문장의 최대 길이:', max(len(i) for i in x_test_rnn))
print('문장의 평균 길이:', sum(len(i) for i in x_train_rnn)//total_data)

훈련 문장의 최대 길이: 2376
테스트 문장의 최대 길이: 1032
문장의 평균 길이: 116


문장의 최대 길이는 200으로 지정하겠습니다.

In [225]:
max_len = 200
x_train_rnn = pad_sequences(x_train_rnn, padding='post', maxlen=max_len)
x_test_rnn = pad_sequences(x_test_rnn, padding='post', maxlen=max_len)

In [226]:
# 레이블 데이터에 대해 원핫인코딩 수행
y_train_rnn = to_categorical(y_train_rnn, num_classes=num_classes)
y_test_rnn = to_categorical(y_test_rnn, num_classes=num_classes)

In [227]:
# validation 데이터셋을 분리하기 위해서 데이터의 개수를 확인합니다.
print(len(x_train_rnn)), print(len(y_train_rnn))

8982
8982


(None, None)

In [228]:
# train, validation 데이터셋 분리
x_train_rnn1 = x_train_rnn[:7000]
x_valid_rnn1 = x_train_rnn[7000:]

y_train_rnn1 = y_train_rnn[:7000]
y_valid_rnn1 = y_train_rnn[7000:]

In [229]:
print(x_train_rnn1.shape)
print(x_valid_rnn1.shape)

print(y_train_rnn1.shape)
print(y_valid_rnn1.shape)

(7000, 200)
(1982, 200)
(7000, 46)
(1982, 46)


In [247]:
vocab_size = len(index_to_word)
print(vocab_size)
word_vector_dim = 126

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim))
model.add(keras.layers.LSTM(126))
model.add(keras.layers.Dense(46, activation='softmax'))
# 모델 훈련
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_rnn1, y_train_rnn1, epochs=50, batch_size=128, validation_data=(x_valid_rnn1, y_valid_rnn1), verbose=1)

30982
Epoch 1/50
55/55 [==============================] - 6s 77ms/step - loss: 3.0339 - accuracy: 0.3212 - val_loss: 2.3569 - val_accuracy: 0.3456
Epoch 2/50
55/55 [==============================] - 4s 64ms/step - loss: 2.3484 - accuracy: 0.3611 - val_loss: 2.3305 - val_accuracy: 0.3552
Epoch 3/50
55/55 [==============================] - 4s 65ms/step - loss: 2.3097 - accuracy: 0.3729 - val_loss: 2.3577 - val_accuracy: 0.3552
Epoch 4/50
55/55 [==============================] - 3s 63ms/step - loss: 2.3361 - accuracy: 0.3697 - val_loss: 2.3878 - val_accuracy: 0.3340
Epoch 5/50
55/55 [==============================] - 4s 64ms/step - loss: 2.3523 - accuracy: 0.3516 - val_loss: 2.2122 - val_accuracy: 0.3739
Epoch 6/50
55/55 [==============================] - 4s 65ms/step - loss: 2.1616 - accuracy: 0.4202 - val_loss: 2.0011 - val_accuracy: 0.4884
Epoch 7/50
55/55 [==============================] - 3s 64ms/step - loss: 2.0223 - accuracy: 0.4828 - val_loss: 2.2928 - val_accuracy: 0.4546
Epoch 8

## 2. 빈도수 상위 5,000개의 단어만 사용

### 데이터 로드

In [248]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=5000, test_split=0.2)

In [249]:
print('훈련 데이터 개수:', len(x_train))
print('데스트 데이터 개수:', len(x_test))

훈련 데이터 개수: 8982
데스트 데이터 개수: 2246


In [250]:
# 데이터 확인
print(x_train[0])
print(x_test[0])

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
[1, 4, 1378, 2025, 9, 697, 4622, 111, 8, 25, 109, 29, 3650, 11, 150, 244, 364, 33, 30, 30, 1398, 333, 6, 2, 159, 9, 1084, 363, 13, 2, 71, 9, 2, 71, 117, 4, 225, 78, 206, 10, 9, 1214, 8, 4, 270, 5, 2, 7, 748, 48, 9, 2, 7, 207, 1451, 966, 1864, 793, 97, 133, 336, 7, 4, 493, 98, 273, 104, 284, 25, 39, 338, 22, 905, 220, 3465, 644, 59, 20, 6, 119, 61, 11, 15, 58, 579, 26, 10, 67, 7, 4, 738, 98, 43, 88, 333, 722, 12, 20, 6, 19, 746, 35, 15, 10, 9, 1214, 855, 129, 783, 21, 4, 2280, 244, 364, 51, 16, 299, 452, 16, 515, 4, 99, 29, 5, 4, 364, 281, 48, 10, 9, 1214, 23, 644, 47, 20, 324, 27, 56, 2, 2, 5, 192, 510, 17, 12]


### 데이터 복원하기

In [251]:
word_index = reuters.get_word_index(path="reuters_word_index.json")
word_index['the']

1

reuters.get_word_index에 +3을 해주어야 학습이 올바르게 이루어질 수 있습니다. 또한 필요한 것은 index:word와 같은 형식이므로 변환해 주겠습니다.

In [252]:
index_to_word = {index+3:word for word, index in word_index.items()}
index_to_word[4]

'the'

index_to_word에서 1,2,3 index는 비어 있으므로 적합한 토큰을 추가해줍니다.

In [253]:
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
    index_to_word[index]=token
print(index_to_word[2])

<unk>


훈련, 테스트 데이터를 텍스트 데이터로 변환하겠습니다.

In [254]:
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)
    
x_train = decoded
print(len(x_train))

8982


In [255]:
x_train[0]

'<sos> <unk> <unk> said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3'

In [256]:
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)
        
x_test = decoded
print(len(x_test))        

2246


In [257]:
x_test[0]

'<sos> the great atlantic and pacific tea co said its three year 345 mln dlr capital program will be be substantially increased to <unk> growth and expansion plans for <unk> inc and <unk> inc over the next two years a and p said the acquisition of <unk> in august 1986 and <unk> in december helped us achieve better than expected results in the fourth quarter ended february 28 its net income from continuing operations jumped 52 6 pct to 20 7 mln dlrs or 55 cts a share in the latest quarter as sales increased 48 3 pct to 1 58 billion dlrs a and p gave no details on the expanded capital program but it did say it completed the first year of the program during 1986 a and p is 52 4 pct owned by lt <unk> <unk> of west germany reuter 3'

### 복원 데이터를 다시 벡터화 하기

In [258]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [259]:
dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

(8982, 4867)


In [260]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

(8982, 4867)


In [261]:
x_test_dtm = dtmvector.transform(x_test)
tfidfv_test = tfidf_transformer.transform(x_test_dtm)
print(tfidfv.shape)

(8982, 4867)


### 나이브 베이즈 분류기(Multinomial Naive Bayes Classifier)

In [262]:
model = MultinomialNB()
model.fit(tfidfv, y_train)

MultinomialNB()

In [263]:
x_test_dtm = dtmvector.transform(x_test) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환

predicted = model.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6731967943009796


### Complement Naive Bayes Classifier(CNB)

In [264]:
cb = ComplementNB()
cb.fit(tfidfv, y_train)

ComplementNB()

In [265]:
predicted = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7707034728406055


### 로지스틱 회귀(Logistic Regression)

In [266]:
lr = LogisticRegression(C=10000, penalty='l2')
lr.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000)

In [267]:
predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8058771148708815


### 선형 서포트 벡터 머신(Linear Support Vector Machine)

In [268]:
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False)
lsvc.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1000, dual=False, max_iter=500, penalty='l1')

In [269]:
predicted = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7595725734639359


### 결정 트리(Decision Tree)

In [270]:
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [271]:
predicted = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6179875333926982


### 랜덤 포레스트(Random Forest)

In [272]:
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [273]:
predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.701246660730187


### 그래디언트 부스팅 트리(GradientBoostingClassifier)

In [274]:
grbt = GradientBoostingClassifier(random_state=0) # verbose=3
grbt.fit(tfidfv, y_train)

GradientBoostingClassifier(random_state=0)

In [275]:
predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.767586821015138


### 보팅(Voting)

In [276]:
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=10000)),
                             ('cb', ComplementNB()),
                             ('grbt',
                              GradientBoostingClassifier(random_state=0))],
                 n_jobs=-1, voting='soft')

In [277]:
predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8161175422974176


### RNN(LSTM)

In [278]:
# 모듈 import

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [279]:
(x_train_rnn, y_train_rnn), (x_test_rnn, y_test_rnn) = reuters.load_data(num_words=5000, test_split=0.2)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [280]:
num_classes = max(y_train_rnn)+1
num_classes

46

In [281]:
len(index_to_word)

30982

In [282]:
vocab_size = len(index_to_word)
tag_size = num_classes

In [283]:
print('단어 사전의 개수:', vocab_size)
print('레이블의 개수:', tag_size)

단어 사전의 개수: 30982
레이블의 개수: 46


In [284]:
total_data = len(x_train_rnn) + len(x_test_rnn)
total_data

11228

In [285]:
total_data = len(x_train_rnn) + len(x_test_rnn)
print('훈련 문장의 최대 길이:', max(len(i) for i in x_train_rnn))
print('테스트 문장의 최대 길이:', max(len(i) for i in x_test_rnn))
print('문장의 평균 길이:', sum(len(i) for i in x_train_rnn)//total_data)

훈련 문장의 최대 길이: 2376
테스트 문장의 최대 길이: 1032
문장의 평균 길이: 116


문장의 최대 길이는 200으로 지정하겠습니다.

In [286]:
max_len = 200
x_train_rnn = pad_sequences(x_train_rnn, padding='post', maxlen=max_len)
x_test_rnn = pad_sequences(x_test_rnn, padding='post', maxlen=max_len)

In [287]:
# 레이블 데이터에 대해 원핫인코딩 수행
y_train_rnn = to_categorical(y_train_rnn, num_classes=num_classes)
y_test_rnn = to_categorical(y_test_rnn, num_classes=num_classes)

In [288]:
# validation 데이터셋을 분리하기 위해서 데이터의 개수를 확인합니다.
print(len(x_train_rnn)), print(len(y_train_rnn))

8982
8982


(None, None)

In [289]:
# train, validation 데이터셋 분리
x_train_rnn1 = x_train_rnn[:7000]
x_valid_rnn1 = x_train_rnn[7000:]

y_train_rnn1 = y_train_rnn[:7000]
y_valid_rnn1 = y_train_rnn[7000:]

In [290]:
print(x_train_rnn1.shape)
print(x_valid_rnn1.shape)

print(y_train_rnn1.shape)
print(y_valid_rnn1.shape)

(7000, 200)
(1982, 200)
(7000, 46)
(1982, 46)


In [291]:
vocab_size = len(index_to_word)
print(vocab_size)
word_vector_dim = 126

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim))
model.add(keras.layers.LSTM(126))
model.add(keras.layers.Dense(46, activation='softmax'))
# 모델 훈련
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_rnn1, y_train_rnn1, epochs=30, batch_size=128, validation_data=(x_valid_rnn1, y_valid_rnn1), verbose=1)

30982
Epoch 1/30
55/55 [==============================] - 6s 75ms/step - loss: 3.0237 - accuracy: 0.3118 - val_loss: 2.3560 - val_accuracy: 0.3552
Epoch 2/30
55/55 [==============================] - 4s 64ms/step - loss: 2.3688 - accuracy: 0.3610 - val_loss: 2.3353 - val_accuracy: 0.3557
Epoch 3/30
55/55 [==============================] - 3s 64ms/step - loss: 2.3665 - accuracy: 0.3529 - val_loss: 2.2910 - val_accuracy: 0.3628
Epoch 4/30
55/55 [==============================] - 4s 67ms/step - loss: 2.2857 - accuracy: 0.3734 - val_loss: 2.3977 - val_accuracy: 0.3461
Epoch 5/30
55/55 [==============================] - 3s 64ms/step - loss: 2.4241 - accuracy: 0.3566 - val_loss: 2.3449 - val_accuracy: 0.3552
Epoch 6/30
55/55 [==============================] - 3s 63ms/step - loss: 2.3242 - accuracy: 0.3688 - val_loss: 2.2866 - val_accuracy: 0.3703
Epoch 7/30
55/55 [==============================] - 4s 64ms/step - loss: 2.1994 - accuracy: 0.3883 - val_loss: 2.4041 - val_accuracy: 0.3572
Epoch 8

## 3. 직접 단어 개수를 설정해서 사용
num_words = 15000

In [292]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=15000, test_split=0.2)

In [293]:
print('훈련 데이터 개수:', len(x_train))
print('데스트 데이터 개수:', len(x_test))

훈련 데이터 개수: 8982
데스트 데이터 개수: 2246


In [294]:
# 데이터 확인
print(x_train[0])
print(x_test[0])

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
[1, 4, 1378, 2025, 9, 697, 4622, 111, 8, 25, 109, 29, 3650, 11, 150, 244, 364, 33, 30, 30, 1398, 333, 6, 2, 159, 9, 1084, 363, 13, 2, 71, 9, 2, 71, 117, 4, 225, 78, 206, 10, 9, 1214, 8, 4, 270, 5, 2, 7, 748, 48, 9, 2, 7, 207, 1451, 966, 1864, 793, 97, 133, 336, 7, 4, 493, 98, 273, 104, 284, 25, 39, 338, 22, 905, 220, 3465, 644, 59, 20, 6, 119, 61, 11, 15, 58, 579, 26, 10, 67, 7, 4, 738, 98, 43, 88, 333, 722, 12, 20, 6, 19, 746, 35, 15, 10, 9, 1214, 855, 129, 783, 21, 4, 2280, 244, 364, 51, 16, 299, 452, 16, 515, 4, 99, 29, 5, 4, 364, 281, 48, 10, 9, 1214, 23, 644, 47, 20, 324, 27, 56, 2, 2, 5, 192, 510, 17, 12]


### 데이터 복원하기

In [295]:
word_index = reuters.get_word_index(path="reuters_word_index.json")
word_index['the']

1

reuters.get_word_index에 +3을 해주어야 학습이 올바르게 이루어질 수 있습니다. 또한 필요한 것은 index:word와 같은 형식이므로 변환해 주겠습니다.

In [296]:
index_to_word = {index+3:word for word, index in word_index.items()}
index_to_word[4]

'the'

index_to_word에서 1,2,3 index는 비어 있으므로 적합한 토큰을 추가해줍니다.

In [297]:
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
    index_to_word[index]=token
print(index_to_word[2])

<unk>


훈련, 테스트 데이터를 텍스트 데이터로 변환하겠습니다.

In [298]:
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)
    
x_train = decoded
print(len(x_train))

8982


In [299]:
x_train[0]

'<sos> <unk> <unk> said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3'

In [300]:
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)
        
x_test = decoded
print(len(x_test))        

2246


In [301]:
x_test[0]

'<sos> the great atlantic and pacific tea co said its three year 345 mln dlr capital program will be be substantially increased to <unk> growth and expansion plans for <unk> inc and <unk> inc over the next two years a and p said the acquisition of <unk> in august 1986 and <unk> in december helped us achieve better than expected results in the fourth quarter ended february 28 its net income from continuing operations jumped 52 6 pct to 20 7 mln dlrs or 55 cts a share in the latest quarter as sales increased 48 3 pct to 1 58 billion dlrs a and p gave no details on the expanded capital program but it did say it completed the first year of the program during 1986 a and p is 52 4 pct owned by lt <unk> <unk> of west germany reuter 3'

### 복원 데이터를 다시 벡터화 하기

In [302]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [303]:
dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

(8982, 14227)


In [304]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

(8982, 14227)


In [305]:
x_test_dtm = dtmvector.transform(x_test)
tfidfv_test = tfidf_transformer.transform(x_test_dtm)
print(tfidfv.shape)

(8982, 14227)


### 나이브 베이즈 분류기(Multinomial Naive Bayes Classifier)

In [306]:
model = MultinomialNB()
model.fit(tfidfv, y_train)

MultinomialNB()

In [307]:
x_test_dtm = dtmvector.transform(x_test) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환

predicted = model.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6331255565449688


### Complement Naive Bayes Classifier(CNB)

In [308]:
cb = ComplementNB()
cb.fit(tfidfv, y_train)

ComplementNB()

In [309]:
predicted = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7720391807658059


### 로지스틱 회귀(Logistic Regression)

In [310]:
lr = LogisticRegression(C=10000, penalty='l2')
lr.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000)

In [311]:
predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8125556544968834


### 선형 서포트 벡터 머신(Linear Support Vector Machine)

In [312]:
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False)
lsvc.fit(tfidfv, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1000, dual=False, max_iter=500, penalty='l1')

In [313]:
predicted = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7836153161175423


### 결정 트리(Decision Tree)

In [314]:
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [315]:
predicted = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6193232413178985


### 랜덤 포레스트(Random Forest)

In [316]:
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [317]:
predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6714158504007124


### 그래디언트 부스팅 트리(GradientBoostingClassifier)

In [318]:
grbt = GradientBoostingClassifier(random_state=0) # verbose=3
grbt.fit(tfidfv, y_train)

GradientBoostingClassifier(random_state=0)

In [319]:
predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7707034728406055


### 보팅(Voting)

In [320]:
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=10000)),
                             ('cb', ComplementNB()),
                             ('grbt',
                              GradientBoostingClassifier(random_state=0))],
                 n_jobs=-1, voting='soft')

In [321]:
predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8165627782724845


### RNN(LSTM)

In [322]:
# 모듈 import

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [323]:
(x_train_rnn, y_train_rnn), (x_test_rnn, y_test_rnn) = reuters.load_data(num_words=15000, test_split=0.2)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [324]:
num_classes = max(y_train_rnn)+1
num_classes

46

In [325]:
len(index_to_word)

30982

In [326]:
vocab_size = len(index_to_word)
tag_size = num_classes

In [327]:
print('단어 사전의 개수:', vocab_size)
print('레이블의 개수:', tag_size)

단어 사전의 개수: 30982
레이블의 개수: 46


In [328]:
total_data = len(x_train_rnn) + len(x_test_rnn)
total_data

11228

In [329]:
total_data = len(x_train_rnn) + len(x_test_rnn)
print('훈련 문장의 최대 길이:', max(len(i) for i in x_train_rnn))
print('테스트 문장의 최대 길이:', max(len(i) for i in x_test_rnn))
print('문장의 평균 길이:', sum(len(i) for i in x_train_rnn)//total_data)

훈련 문장의 최대 길이: 2376
테스트 문장의 최대 길이: 1032
문장의 평균 길이: 116


문장의 최대 길이는 200으로 지정하겠습니다.

In [330]:
max_len = 200
x_train_rnn = pad_sequences(x_train_rnn, padding='post', maxlen=max_len)
x_test_rnn = pad_sequences(x_test_rnn, padding='post', maxlen=max_len)

In [331]:
# 레이블 데이터에 대해 원핫인코딩 수행
y_train_rnn = to_categorical(y_train_rnn, num_classes=num_classes)
y_test_rnn = to_categorical(y_test_rnn, num_classes=num_classes)

In [332]:
# validation 데이터셋을 분리하기 위해서 데이터의 개수를 확인합니다.
print(len(x_train_rnn)), print(len(y_train_rnn))

8982
8982


(None, None)

In [333]:
# train, validation 데이터셋 분리
x_train_rnn1 = x_train_rnn[:7000]
x_valid_rnn1 = x_train_rnn[7000:]

y_train_rnn1 = y_train_rnn[:7000]
y_valid_rnn1 = y_train_rnn[7000:]

In [334]:
print(x_train_rnn1.shape)
print(x_valid_rnn1.shape)

print(y_train_rnn1.shape)
print(y_valid_rnn1.shape)

(7000, 200)
(1982, 200)
(7000, 46)
(1982, 46)


In [335]:
vocab_size = len(index_to_word)
print(vocab_size)
word_vector_dim = 126

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim))
model.add(keras.layers.LSTM(126))
model.add(keras.layers.Dense(46, activation='softmax'))
# 모델 훈련
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_rnn1, y_train_rnn1, epochs=30, batch_size=128, validation_data=(x_valid_rnn1, y_valid_rnn1), verbose=1)

30982
Epoch 1/30
55/55 [==============================] - 6s 75ms/step - loss: 2.9824 - accuracy: 0.3164 - val_loss: 2.3524 - val_accuracy: 0.3552
Epoch 2/30
55/55 [==============================] - 3s 63ms/step - loss: 2.3570 - accuracy: 0.3667 - val_loss: 2.3462 - val_accuracy: 0.3496
Epoch 3/30
55/55 [==============================] - 4s 64ms/step - loss: 2.3628 - accuracy: 0.3663 - val_loss: 2.3546 - val_accuracy: 0.3466
Epoch 4/30
55/55 [==============================] - 3s 64ms/step - loss: 2.3543 - accuracy: 0.3694 - val_loss: 2.4140 - val_accuracy: 0.3552
Epoch 5/30
55/55 [==============================] - 4s 64ms/step - loss: 2.3729 - accuracy: 0.3598 - val_loss: 2.3362 - val_accuracy: 0.3547
Epoch 6/30
55/55 [==============================] - 3s 63ms/step - loss: 2.3284 - accuracy: 0.3599 - val_loss: 2.3280 - val_accuracy: 0.3597
Epoch 7/30
55/55 [==============================] - 4s 64ms/step - loss: 2.3306 - accuracy: 0.3719 - val_loss: 2.2695 - val_accuracy: 0.3587
Epoch 8